# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [34]:
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

from collections import deque  # for memory
from tqdm import tqdm          # for progress bar

How the game looks (without our agent):

In [2]:
env = gym.make('CartPole-v1', render_mode='human')
for _ in tqdm(range(10)):
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
env.close()

100%|██████████| 10/10 [00:09<00:00,  1.07it/s]


## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}

However, we don't know $Q^*$. So, we use neural network as a approximators, we can simply create one and train it to resemble $Q^*$.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}

The difference between the two sides of the equality is known as the temporal difference error, $\delta$:

\begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align}

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.

In [35]:
class Model(nn.Module):
    def __init__(self, observation_size, action_size):
        super(Model, self).__init__()
#         # initialise layers here
        self.dense1 = nn.Linear(observation_size, 100)
        self.dense2 = nn.Linear(100, action_size)

    def forward(self, x):
        # send x through the network
        x = self.dense1(x)
        x = F.relu(x)
        x = self.dense2(x)        
        return x

    def predict(self, x):
        x = self.forward(x)
        return torch.argmax(x)


### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [36]:
from multiprocessing.sharedctypes import Value
from random import random
from tkinter import Variable


class Agent:
    def __init__(self, observation_size, action_size):
        self.observation_size = observation_size
        self.action_size = action_size
        self.model = Model(observation_size, action_size)
        self.discount_value = 0.1
        self.epsilon = 1.0
        self.epsilonDecay = 0.99
        self.epsilonMin = 0.00
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.memorySize = 1000
        self.memory = deque([], self.memorySize)
        #self.memory = deque(self.memorySize) # memory that stores N most new transitions
        # good place to store hyperparameters as attributes

    # add to memory
    #def remember(self, state, action, reward, next_state, done):
    def remember(self, transition):
        self.memory.append(transition)
        #if (self.memory.count >= self.memorySize):
        #    self.memory.popleft()
        #self.memory.append((state, action, reward, next_state, done))
        
    # return an action from the model
    def act(self, state):
        if random.uniform(0, 1) < self.epsilon:
            action = env.action_space.sample()
        else:
            action = self.model.predict(state)
        
        transition = env.step(action)
        self.remember(transition)

        state = torch.tensor(state, dtype=torch.float32)
        
        #return self.model.predict(state)
        action = self.model.predict(state)
        return int(action)
        
        
    # update model based on replay memory
    # you might want to make a self.train() helper method
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        self.optimizer.zero_grad() # clean
        for element in minibatch:
            self.train(element)

        self.epsilon *= self.epsilonDecay
        self.optimizer.step()
    

    def train(self, transition):
        s0, a, r, s1, d = transition
        pred = self.model.predict(torch.tensor(s0))
        if not d:
            #value = self.discount_value * torch.argmax(self.act(s0))
            #value = self.discount_value * self.model.predict(s1)
            value = r + self.discount_value * float(torch.max(self.model.forward(s1)))
        else:
            value = r
        pred = Variable(torch.tensor(pred), requires_grad = True)
        reward = Variable(torch.tensor(value))
        loss = self.criterion(pred, reward)
        loss.backward()
        self.optimizer.step()


### Main Training loop
---
Make a function that takes and environment and an agent, and runs through $n$ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [37]:
def train(env, agent, episodes=1000, batch_size=64):  # train for many games
    for _ in tqdm(range(episodes)):
        state, _ = env.reset()
        done = False
        while not done:
            # 1. make a move in game.
            action = agent.act(state)
            next_state, reward, done, _, _, = env.step(action)
            # 2. have the agent remember stuff.
            #agent.remember()
            # 3. update state
            state = next_state
            # 4. if we have enough experiences in out memory, learn from a batch with replay.
            if len(agent.memory) >= batch_size:
                agent.replay(batch_size)
    env.close()

### Putting it together
---
We train an agent on the environment:


In [38]:
env = gym.make('CartPole-v1')  # , render_mode='human')
agent = Agent(env.observation_space.shape[0], env.action_space.n)
train(env, agent)
torch.save(agent.model.state_dict(), 'model.pth')

  0%|          | 0/1000 [00:00<?, ?it/s]


AttributeError: 'builtin_function_or_method' object has no attribute 'uniform'

## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/